In [1]:
# Imports
import numpy as np

In [2]:
# System parameters
D = 17  # ps/nm/km
gamma = 1.3e-3  # 1/W/m
B_signal = 30e9  # signal bandwidth = channel spacing for Nyquist WDM [Hz]
N_channel = 17  # numbers of channels
NFdB = 4  # amplifier noise figure [dB]
Length=100e3  # span length (m)
alpha=0.2e-3  # fibre attenuation (dB/m)
N_spans=10  # number of spans
N_pol=2  # number of polarizations

In [3]:
# Constants
h = 6.6256e-34  # Planck's constant [J/s]
nu = 299792458 / 1550e-9  # light frequency [Hz]
SNR2OSNR = 10 * np.log10( B_signal / 12.5e9 * N_pol / 2 )  # Eq. (34) of Essiambre et al., "Capacity Limits of Optical Fiber Networks," J. Light. Technol., vol. 28, no. 4, pp. 662-701, Feb. 2010.
dB2Neper = 10 / np.log(10)

In [4]:
# Set launch power per channel
vec_start = -10
vec_stop = 10
vec_step = .25
powerVec = np.linspace(-10, 10, num=int(( vec_stop-vec_start) / vec_step + 1 ))  # launch power [dBm]

In [5]:
# Some more quantities
B_total = N_channel * B_signal  # total system bandwidth
beta2 = -np.square(1550e-9) * ( D * 1e-6 ) / ( 2 * np.pi * 3e8)  # propagation constant
GaindB = Length * alpha  # amplifier gain (dB)
L_eff = (( 1 - np.exp ( -( alpha / dB2Neper ) * Length )) / (alpha / dB2Neper))  # effective length [m]
L_effa = 1 / (alpha/dB2Neper)  # asymptotic effective length [m]
G_tx_ch = np.divide( np.power( 10, ( powerVec - 30 ) / 10 ) , B_signal) # [W/Hz]
ASE = N_pol * N_spans * np.power(10, NFdB / 10 ) / 2 * ( np.power(10, GaindB / 10 ) - 1 ) * h * nu  # Eq. (50)

In [25]:
# GN model
epsilon = 3 / 10 * np.log( 1 + 6 / Length * L_effa / ( np.arcsinh( 0.5 * np.square(np.pi) * np.abs( beta2 ) * L_effa * np.square( B_total ))))  # Eq. (40)
G_NLI = np.multiply( np.square(gamma) , np.power( G_tx_ch, 3 ) * np.square(L_eff) * np.power( 2/3, 3 ) * np.arcsinh( 0.5 * np.square( np.pi ) * np.abs( beta2 ) * L_effa * np.square(B_total))/( np.pi * np.abs( beta2 ) * L_effa))  # Eq. (36)

In [26]:
# SNR calculation
GN = {}

GN['SNR_NLI'] = 10 * np.log10( np.divide( G_tx_ch , ( ASE + np.power( N_spans, ( 1 + epsilon )) * G_NLI) ))  # Eq. (16), (22)
GN['SNR_ASE'] = 10 * np.log10( np.divide( G_tx_ch, ASE ))

In [23]:
# OSNR calculation
GN['OSNR_NLI'] = np.add(GN['SNR_NLI'], SNR2OSNR)
GN['OSNR_EDFA'] = np.add(GN['SNR_ASE'], SNR2OSNR)

GN['power'] = powerVec

In [24]:
for key, item in GN.items():
    print("{}:\n{}".format(key, item))

SNR_NLI:
[10.18899756 10.43781182 10.68640298 10.93472917 11.18274067 11.43037853
 11.67757279 11.92424051 12.17028341 12.41558508 12.6600077  12.90338828
 13.14553418 13.38621799 13.62517162 13.86207941 14.09657045 14.32820975
 14.5564885  14.78081323 15.00049423 15.21473315 15.42261048 15.62307318
 15.81492354 15.99681012 16.16722241 16.32449078 16.4667938  16.59217487
 16.69857041 16.78385073 16.84587455 16.88255614 16.89194272 16.8722975
 16.82218227 16.74053179 16.62671252 16.48055904 16.30238364 16.09295817
 15.8534702  15.5854586  15.29073582 14.97130441 14.62927509 14.26679218
 13.88597015 13.48884328 13.07732887 12.65320279 12.21808594 11.7734394
 11.32056638 10.86061884 10.39460731  9.92341256  9.44779805  8.96842249
  8.48585198  8.00057146  7.51299513  7.02347594  6.53231408  6.0397644
  5.54604299  5.0513328   4.55578862  4.05954127  3.56270124  3.06536181
  2.56760167  2.06948716  1.57107425  1.07241006  0.57353432  0.0744805
 -0.42472323 -0.92405314 -1.42348925]
SNR_ASE: